In [ ]:
import pypsa
import yaml
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import cartopy.crs as ccrs
import cartopy

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors



plt.style.use(['bmh', 'matplotlibrc'])
xr.set_options(display_style='html')

%matplotlib inline

In [ ]:
path = "../../../playgrounds/pr/"
clusters = 181
output = "../results/graphics"
resources = "resources" # "20211007-resources"

In [ ]:
with open(path + "pypsa-eur-sec/config.yaml") as file:
    config = yaml.safe_load(file)

In [ ]:
# shapes
nodes = gpd.read_file(path + f"pypsa-eur/{resources}/regions_onshore_elec_s_{clusters}.geojson").set_index('name')
cts = gpd.read_file(path + f"pypsa-eur/{resources}/country_shapes.geojson").set_index('name')

regions = gpd.read_file(path + f"pypsa-eur/{resources}/regions_onshore.geojson").append(
          gpd.read_file(path + f"pypsa-eur/{resources}/regions_offshore.geojson"))
regions = regions.dissolve('name') 
onregions = gpd.read_file(path + f"pypsa-eur/{resources}/regions_onshore.geojson").set_index('name')
regions["Area"] = regions.to_crs(epsg=3035).area.div(1e6)
onregions["Area"] = onregions.to_crs(epsg=3035).area.div(1e6)
nodes["Area"] = nodes.to_crs(epsg=3035).area.div(1e6)

### Plots

In [ ]:
co2 = pd.read_csv(path + "pypsa-eur-sec/resources/co2_totals.csv", index_col=0)

fig, ax = plt.subplots(figsize=(7,6))
co2.plot.barh(ax=ax, stacked=True, cmap='tab20')
plt.savefig(output + "/co2.pdf")

### Industrial Production Fuel Switching

In [ ]:
iproduction_today = pd.read_csv(path + "pypsa-eur-sec/resources/industrial_production_per_country.csv", index_col=0).sum()
iproduction_tomorrow = pd.read_csv(path + "pypsa-eur-sec/resources/industrial_production_per_country_tomorrow_2030.csv", index_col=0).sum()
iratios = pd.read_csv(path + "pypsa-eur-sec/resources/industry_sector_ratios.csv", index_col=0)
ienergy_today = pd.read_csv(path + "pypsa-eur-sec/resources/industrial_energy_demand_per_country_today.csv", index_col=0, header=[0,1]).sum(level=1, axis=1)

In [ ]:
iproduction_today

In [ ]:
ienergy_today.rename({"gas": "methane"}, inplace=True)

In [ ]:
iratios.rename({"elec": "electricity", "naphtha": "liquid"}, inplace=True)
iratios.loc["solid",:] = iratios.loc[["coke", "coal"]].sum()
iratios.drop(["coke", "coal", "process emission", "process emission from feedstock"], axis=0, inplace=True)

In [ ]:
ienergy_tomorrow = iratios * iproduction_tomorrow / 1e3
bc = ["Chlorine", "HVC", "HVC (mechanical recycling)", "HVC (chemical recycling)", "Methanol"]

In [ ]:
ienergy_tomorrow["Basic chemicals (without ammonia)"] = ienergy_tomorrow[bc].sum(axis=1)
ienergy_tomorrow.drop(bc, axis=1, inplace=True)
ienergy_today["DRI + Electric arc"] = 0.
ienergy_today.loc["hydrogen",:] = 0.
ienergy_tomorrow.loc["other",:] = 0.
ienergy_tomorrow.loc["waste",:] = 0.

In [ ]:
ienergy_tomorrow.sort_index(axis=1, inplace=True)
ienergy_today.sort_index(axis=1, inplace=True)

ienergy_tomorrow.sort_index(axis=0, inplace=True)
ienergy_today.sort_index(axis=0, inplace=True)

In [ ]:
tech_colors = config["plotting"]["tech_colors"]

In [ ]:
tech_colors["electricity"] = '#ace37f'
tech_colors["hydrogen"] = '#f073da'

In [ ]:
fig, ax = plt.subplots(figsize=(4,6))

ienergy_tomorrow.T.plot.barh(ax=ax, stacked=True, width=.3, color=tech_colors, position=1, edgecolor='k')
ienergy_today.T.plot.barh(ax=ax, stacked=True, width=.3, color=tech_colors, position=0, edgecolor='k')


plt.xlabel("Final energy and non-energy [TWh/a]")
plt.xlim(-50, 1300)
plt.ylim(-1,23)
handles, labels = ax.get_legend_handles_labels()
n = ienergy_today.shape[0]
plt.legend(handles[:n], labels[:n])
plt.savefig(output + "/fec_industry_today_tomorrow.pdf", bbox_inches='tight')

In [ ]:
df = ienergy_tomorrow.sum(axis=1).sort_values(ascending=False)
df = df.loc[df>0]

In [ ]:
fig, ax = plt.subplots(figsize=(3,4))

df.plot.bar(ax=ax, width=.3, edgecolor='k')

plt.ylabel("Final energy and non-energy [TWh/a]")
plt.xlabel("")

plt.savefig(output + "/fec_industry_tomorrow_by_carrier.pdf", bbox_inches='tight')

## Process Emissions

In [ ]:
pe = pd.read_csv(path + "pypsa-eur-sec/resources/industry_sector_ratios.csv", index_col=0).filter(like='process emission', axis=0).sum()

In [ ]:
pe_today = iproduction_today * pe / 1e3

In [ ]:
pe_tomorrow = iproduction_tomorrow * pe / 1e3

In [ ]:
fig, ax = plt.subplots(figsize=(4,6))

pe_tomorrow.plot.barh(ax=ax, width=.3, color='peachpuff', position=1, edgecolor='k')
pe_today.plot.barh(ax=ax, width=.3, color='indianred', position=0, edgecolor='k')

plt.xlabel(f"Process emissions [MtCO$_2$/a]")
plt.xlim(-4, 110)
plt.ylim(-1,27)
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles[:n], [f"2050:  {pe_tomorrow.sum():.0f} MtCO$_2$/a", f"today: {pe_today.sum():.0f} MtCO$_2$/a"])
plt.savefig(output + "/process-emissions.pdf", bbox_inches='tight')

### Power Plants

In [ ]:
df = pd.read_csv(path + "pypsa-eur/resources/powerplants.csv", index_col=0)

df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")

df.Fueltype = df.Fueltype.str.strip().replace("CCGT, Thermal", "CCGT")

df = df.loc[~df.Fueltype.isin(['Other', 'Pv', 'Storage Technologies'])]

colors = {
    "Bioenergy": '#80c944',
    "CCGT": '#a85522',
    "Geothermal": 'red',
    "Hard Coal": 'black',
    "Lignite": '#826837',
    "Hydro": "#235ebc",
    "Nuclear": "#ff8c00",
    "OCGT": '#e0986c',
    "Oil": '#c9c9c9',
    "Waste": "purple",
}

crs = ccrs.AlbersEqualArea()

df = df.cx[-12:30, 35:72]
df = df.to_crs(crs.proj4_init)

fig, ax = plt.subplots(figsize=(9.5,9.5), subplot_kw={"projection": crs})

ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

df.plot(
    ax=ax,
    column='Fueltype',
    markersize=df['Capacity']/35,
    alpha=0.75,
    legend=True,
    cmap=mcolors.ListedColormap(pd.Series(df.Fueltype.unique()).sort_values().map(colors).values),
    legend_kwds=dict(title="Technology (size ~ capacity)", frameon=False)
)



plt.gca().outline_patch.set_visible(False)
ax.set_facecolor('white')
plt.savefig(output + "/powerplants.pdf", bbox_inches='tight')

### Hotmaps

In [ ]:
def prepare_hotmaps_database():
    """
    Load hotmaps database of industrial sites.
    """

    df = pd.read_csv(path + "pypsa-eur-sec/data/Industrial_Database.csv", sep=";", index_col=0)

    df[["srid", "coordinates"]] = df.geom.str.split(';', expand=True)

    # remove those sites without valid locations
    df.drop(df.index[df.coordinates.isna()], inplace=True)

    df['coordinates'] = gpd.GeoSeries.from_wkt(df['coordinates'])

    gdf = gpd.GeoDataFrame(df, geometry='coordinates', crs="EPSG:4326")

    return gdf

In [ ]:

crs = ccrs.AlbersEqualArea()

hotmaps = prepare_hotmaps_database()
hotmaps = hotmaps.cx[-12:30, 35:72]
hotmaps = hotmaps.to_crs(crs.proj4_init)

fig, ax = plt.subplots(figsize=(9.5,9.5), subplot_kw={"projection": crs})

ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

hotmaps.plot(
    ax=ax,
    column='Subsector',
    markersize=hotmaps['Emissions_ETS_2014']/3e4,
    alpha=0.5,
    legend=True,
    #transform=ccrs.epsg(3395),
    legend_kwds=dict(title="Industry Sector (size ~ emissions)", frameon=False)
)



plt.gca().outline_patch.set_visible(False)
ax.set_facecolor('white')

plt.savefig(output + "/hotmaps.pdf", bbox_inches='tight')

### Salt Caverns

In [ ]:
cavern_nodes = pd.read_csv("../../../playgrounds/pr/pypsa-eur-sec/resources/salt_cavern_potentials_s_181.csv", index_col=0)
cavern_nodes = cavern_nodes.where(cavern_nodes>0.5)

In [ ]:
cavern_regions = gpd.read_file("../../../playgrounds/pr/pypsa-eur/resources/regions_onshore_elec_s_181.geojson").set_index('name')

In [ ]:
cavern_offregions = gpd.read_file("../../../playgrounds/pr/pypsa-eur/resources/regions_offshore_elec_s_181.geojson").set_index('name')

In [ ]:
cavern_regions.to_crs(crs.proj4_init).total_bounds

In [ ]:
def plot_salt_caverns_by_node(
    cavern_nodes,
    cavern_regions,
    storage_type='onshore',
    cmap = "GnBu",
    vmin = 1,
    vmax = 3000,
    fn = None,
    label = r"H$_2$ Storage Potential [TWh]",
):

    crs = ccrs.EqualEarth()

    cavern_regions = cavern_regions.to_crs(crs.proj4_init)
    
    fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": crs})

    cavern_regions.plot(
        ax=ax,
        column=cavern_nodes[storage_type].reindex(cavern_regions.index),
        #transform=ccrs.PlateCarree(),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmin=vmin,
        vmax=vmax,
        legend_kwds={
            'label': label,
            "shrink": 0.7,
            "extend": "max",
        },
        norm=mcolors.LogNorm(vmin=1, vmax=vmax) 
    )

    plt.title(f"{storage_type.capitalize()} Salt Cavern H$_2$ Storage Potentials")
    
    plt.xlim(-1e6, 2.6e6)
    plt.ylim(4.3e6, 7.8e6)

    #plt.xlim(-14.5, 31.5)
    #plt.ylim(34, 72)

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor('white')

    if fn is None:
        plt.savefig(f"{output}/cavern-potentials-{storage_type}.pdf", bbox_inches='tight')

In [ ]:
plot_salt_caverns_by_node(cavern_nodes, cavern_regions, storage_type='onshore')

In [ ]:
plot_salt_caverns_by_node(cavern_nodes, cavern_regions, storage_type='nearshore')

In [ ]:
plot_salt_caverns_by_node(cavern_nodes, cavern_offregions, storage_type='offshore')

In [ ]:
caverns = gpd.read_file("../../../playgrounds/pr/pypsa-eur-sec/data/h2_salt_caverns_GWh_per_sqkm.geojson")

crs = ccrs.EqualEarth()

caverns = caverns.to_crs(crs.proj4_init)

fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": crs})

ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

caverns.plot(
    ax=ax,
    column='storage_type',
    cmap="tab10_r", # "tab10_r",
    legend=True,
    linewidth=0,
    #transform=ccrs.epsg(3395),
    legend_kwds=dict(title="Salt Caverns for\nHydrogen Storage", frameon=False, loc=(0.2,.8))
)


plt.xlim(-1e6, 2.6e6)
plt.ylim(4.3e6, 7.8e6)

plt.gca().outline_patch.set_visible(False)
ax.set_facecolor('white')

plt.savefig(output + "/caverns.pdf", bbox_inches='tight')

## Biomass potentials

In [ ]:
bio = pd.read_csv(path + f"pypsa-eur-sec/resources/biomass_potentials_s_{clusters}.csv", index_col=0).div(1e6) # TWh/a

In [ ]:
def plot_biomass_potentials(bio, nodes, kind, fn=None):

    crs = ccrs.EqualEarth()
    nodes = nodes.to_crs(crs.proj4_init)

    fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": crs})

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    nkind = "disregarded biomass" if kind == "not included" else kind
    label = f"{nkind} potentials [TWh/a]"

    nodes.plot(
        ax=ax,
        column=bio[kind],
        cmap="Greens", # "tab10_r",
        legend=True,
        linewidth=0,
        legend_kwds={
            'label': label,
            "shrink": 0.7,
            "extend": "max",
        },
    )
    
    total = bio[kind].sum()

    ax.text(-.8e6, 7.4e6, f'total: {total:.0f} TWh/a', fontsize=15, color='#343434')

    plt.xlim(-1e6, 2.6e6)
    plt.ylim(4.3e6, 7.8e6)

    plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor('white')
    
    if fn is None:
        plt.savefig(output + f"/biomass-{kind}.pdf", bbox_inches='tight')

In [ ]:
for kind in bio.columns:
    plot_biomass_potentials(bio, nodes, kind)

## Ammonia Production

In [ ]:
df = pd.read_csv(path + f"pypsa-eur-sec/resources/ammonia_production.csv", index_col=0) # kt/a

In [ ]:
def plot_ammonia_production(df, cts, year, fn=None):
    
    year = str(year)

    crs = ccrs.EqualEarth()
    cts = cts.to_crs(crs.proj4_init)

    fig, ax = plt.subplots(figsize=(7,7), subplot_kw={"projection": crs})

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    cts.plot(
        ax=ax,
        column=df[year].reindex(cts.index),
        cmap="Purples", # "tab10_r",
        legend=True,
        linewidth=0,
        legend_kwds={
            'label': "Ammonia Production [kt]",
            "shrink": 0.7,
        },
    )
    
    total = df[year].sum()
    ax.text(-.85e6, 7.4e6, f'total: {total:.0f} kt ({year})', fontsize=12, color='#343434')

    plt.xlim(-1e6, 2.6e6)
    plt.ylim(4.3e6, 7.8e6)

    plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor('white')
    
    if fn is None:
        plt.savefig(output + f"/ammonia-{year}.pdf", bbox_inches='tight')

In [ ]:
plot_ammonia_production(df, cts, 2017)

In [ ]:
cts

### Gas Network

In [ ]:
from shapely import wkt

In [ ]:
import sys
path = "../../../playgrounds/pr/pypsa-eur-sec/"
sys.path.append(path + "scripts/")
from build_gas_input_locations import build_gas_input_locations, load_bus_regions

In [ ]:
lng_fn = path + "data/gas_network/scigrid-gas/data/IGGIELGN_LNGs.geojson"
entry_fn = path + "data/gas_network/scigrid-gas/data/IGGIELGN_BorderPoints.geojson"
prod_fn = path + "data/gas_network/scigrid-gas/data/IGGIELGN_Productions.geojson"
planned_lng_fn = path + "data/gas_network/planned_LNGs.csv"

regions_onshore=path + "../pypsa-eur/resources/regions_onshore_elec_s_181.geojson"
regions_offshore= path + "../pypsa-eur/resources/regions_offshore_elec_s_181.geojson"

In [ ]:
regions = load_bus_regions(
    regions_onshore,
    regions_offshore
)

# add a buffer to eastern countries because some
# entry points are still in Russian or Ukrainian territory.
buffer = 9000 # meters
eastern_countries = ['FI', 'EE', 'LT', 'LV', 'PL', 'SK', 'HU', 'RO']
add_buffer_b = regions.index.str[:2].isin(eastern_countries)
regions.loc[add_buffer_b] = regions[add_buffer_b].to_crs(3035).buffer(buffer).to_crs(4326)

countries = regions.index.str[:2].unique().str.replace("GB", "UK")

In [ ]:
pts = build_gas_input_locations(
    lng_fn,
    planned_lng_fn,
    entry_fn,
    prod_fn,
    countries,
)

In [ ]:
sums = pts.groupby('type').p_nom.sum()

for t in ['lng', 'production', 'pipeline']:
    pts.loc[pts['type'] == t, "p_nom"] /= sums[t]

In [ ]:
pts["type"] = pts["type"].replace(dict(production='Fossil Extraction', lng='LNG Terminal', pipeline="Entrypoint"))

In [ ]:
df = pd.read_csv(path + "resources/gas_network.csv", index_col=0)
for col in ["geometry"]:
    df[col] = df[col].apply(wkt.loads)

df = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

In [ ]:
crs = ccrs.AlbersEqualArea()

df = df.to_crs(crs.proj4_init)
pts = pts.to_crs(crs.proj4_init)

fig, ax = plt.subplots(figsize=(9.5,9.5), subplot_kw={"projection": crs})

ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2) 
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

df.plot(
    ax=ax,
    column=df['p_nom'].div(1e3),
    linewidths=df['p_nom'].clip(upper=50e3).div(2e4),
    cmap="Spectral_r",
    vmax=60,
    legend=True,
    legend_kwds=dict(label="Gas Pipeline Capacity [GW]", shrink=0.7, extend='max')
)

pts.plot(
    ax=ax,
    column='type',
    markersize=pts['p_nom'] * 300,
    legend=True,
)


plt.gca().outline_patch.set_visible(False)
ax.set_facecolor('white')
plt.savefig(output + "/gas_network.pdf", bbox_inches='tight')